In [1]:
from datetime import datetime

In [12]:
%run 'utils.ipynb'
init_datadir('/Users/shaayaansayed/Documents/Projects/stockx_scraper/shoes_data/')

In [65]:
path = '/Users/shaayaansayed/Documents/Projects/stockx_scraper/shoes_data/adidas-yeezy-boost-350-v2-cream-white'
with open('{}.json'.format(path)) as f :
    shoe_data = json.load(f)

In [66]:
eastern = timezone('US/Eastern')
central = timezone('US/Central')

def convert2dt_sales(dt_str) :
    return eastern.localize(datetime.strptime(dt_str[:-6], "%Y-%m-%d %H:%M:%S"))

def convert2dt_timestamp(dt_str) :
#     return pytz.utc.localize(datetime.strptime(dt_str, "%Y-%m-%d %H:%M:%S.%f"))
    return eastern.localize(datetime.strptime(dt_str[:-6], "%Y-%m-%d %H:%M:%S.%f"))

sales_flatten = {'size': [], 'price': [], 'timestamp': [], 'sale_timestamp': []}
for sale in shoe_data['sales'] :
    sales_flatten['size'].append(float(sale['size']))
    sales_flatten['price'].append(int(sale['price'][1:]))
    sales_flatten['timestamp'].append(convert2dt_timestamp(sale['timestamp']))
    sales_flatten['sale_timestamp'].append(convert2dt_sales(sale['sale_timestamp']))

asks_flatten = {'size': [], 'price': [], 'avail': [], 'timestamp': []}
for ask in shoe_data['asks'] :
    asks_flatten['size'].append(float(ask['size']))
    asks_flatten['price'].append(int(ask['price'][1:].replace(',', '')))
    asks_flatten['avail'].append(int(ask['avail']))
    asks_flatten['timestamp'].append(convert2dt_timestamp(ask['timestamp']))
    
bids_flatten = {'size': [], 'price': [], 'avail': [], 'timestamp': []}
for bid in shoe_data['bids'] :
    bids_flatten['size'].append(float(bid['size']))
    bids_flatten['price'].append(int(bid['price'][1:].replace(',', '')))
    bids_flatten['avail'].append(int(ask['avail']))
    bids_flatten['timestamp'].append(convert2dt_timestamp(bid['timestamp']))

In [67]:
sales_df = pd.DataFrame(sales_flatten)
asks_df = pd.DataFrame(asks_flatten)
bids_df = pd.DataFrame(bids_flatten)

In [68]:
print(len(sales_df))

740


In [69]:
def get_window(sale_timestamp, timestamps) :
    for i, tmp in enumerate(timestamps) :
        if tmp > sale_timestamp :
            if i == 0 :
                return None
            else :
                return timestamps[i-1], timestamps[i]

stockx_sale = 0
timestamps = sorted(bids_df['timestamp'].unique().tolist())
for ix in range(len(sales_df)) :
    sale = sales_df.iloc[ix]
    
    for df in [asks_df, bids_df] :
        window = get_window(sale['timestamp'], sorted(df['timestamp'].unique().tolist()))
        if window is None :
            continue 
        else :
            before, after = window 
            exists_presale = df[(df['price'] == sale['price']) & (df['size'] == sale['size']) & \
                                   (df['timestamp'] == before)]
            exists_postsale = df[(df['price'] == sale['price']) & (df['size'] == sale['size']) & \
                                   (df['timestamp'] == after)]
            
            if len(exists_presale) > 0 :
                if len(exists_postsale) == 0 :
                    stockx_sale = stockx_sale + 1
                elif int(exists_presale['avail']) > int(exists_postsale['avail']) :
                    stockx_sale = stockx_sale + 1

In [70]:
stockx_sale/215.0

0.06511627906976744

In [49]:
sale = sales_df.iloc[500]
before, after = get_window(sale['timestamp'], timestamps)
k = bids_df[(bids_df['price'] == sale['price']) & (bids_df['size'] == sale['size']) & \
                                   (bids_df['timestamp'] == before)]

In [50]:
k

,avail,price,size,timestamp


In [4]:
# drop sales in 37 minute gap since the bid/ask cannot be verified 

timestamps = bids_df['timestamp'].unique()
diff = np.delete(timestamps, [0]) - np.delete(timestamps, [len(timestamps) - 1])
diff = [int(td.total_seconds()/60) for td in diff]

In [5]:
ix = diff.index(52)

drop = sales_df[(sales_df.index >= timestamps[25]) & (sales_df.index <= timestamps[26])].reset_index()
sales_df = sales_df.reset_index()
sales_df = sales_df[sales_df.apply(lambda x : x.values.tolist() not in drop.values.tolist(), axis=1)]

In [6]:
sales_df = sales_df.reset_index()
def get_tmp_window(sale_tmp, timestamps) :
    for i in range(len(timestamps)) :
        if sale_tmp < timestamps[i] :
            if i == 0 :
                return None 
            else :
                return timestamps[i-1], timestamps[i] 

stockx_sale = 0
timestamps = bids_df['timestamp'].unique()
for ix in range(len(sales_df)) :
    sale = sales_df.iloc[ix]
    window = get_tmp_window(sale['timestamp'], timestamps)
    
    if window is None :
        continue 
    else :
        presale_tmp, postsale_tmp = window
        for df in [bids_df, asks_df] :
            
            exists_presale = ( (df['price'] == sale['price']) & (df['size'] == sale['size']) \
                                  & (df['timestamp'] == presale_tmp) ).any()
            exists_postsale = ((df['price'] == sale['price']) & (df['size'] == sale['size']) \
                                  & (df['timestamp'] == postsale_tmp)).any()
            
            if exists_presale and (not exists_postsale) :
                stockx_sale = stockx_sale + 1
                break

In [7]:
stockx_sale

0

In [109]:
with open('./data/volume_data/adidas-yeezy-boost-350-v2-cream-white.json') as f :
    data = json.load(f)

In [112]:
all_sales = [sale for bar in data['history'] for sale in bar['sales']]
print(len(all_sales))
assert(len(sales_df) == len(all_sales))

def exists(sale, bidask) :
    for entry in bidask :
        if entry['price'] == sale['price'] and entry['size'] == sale['size'] :
            return True
    return False 

bar_bidasks = {convert2dt_timestamp(bar['timestamp']):{'bids':bar['bids'], 'asks':bar['asks']} for bar in data['history']}

stockx_sale = 0
for sale in all_sales :
    sale_dt = convert2dt_sales(sale['date'], sale['time'])
    window = get_tmp_window(sale_dt, sorted(bar_bidasks.keys()))
    
    if window is None :
        continue 
    else :
        presale_tmp, postsale_tmp = window 
        bid_exists_presale = exists(sale, bar_bidasks[presale_tmp]['bids'])
        bid_exists_postsale = exists(sale, bar_bidasks[postsale_tmp]['bids'])
        
        if bid_exists_presale and (not bid_exists_postsale) :
            stockx_sale = stockx_sale + 1
            continue 

        ask_exists_presale = exists(sale, bar_bidasks[presale_tmp]['asks'])
        ask_exists_postsale = exists(sale, bar_bidasks[postsale_tmp]['asks'])
        
        if ask_exists_presale and (not ask_exists_postsale) :
            stockx_sale = stockx_sale + 1

1024
